In [3]:
import pandas 

kinetics = pandas.read_csv( 'kinetics.csv' )
thermal = pandas.read_csv( 'thermal.csv' ) 

print( kinetics.columns )
print( thermal.columns ) 

Index(['well', 'rate', 'mutant', 'protein_yield', 'dilution', 'note'], dtype='object')
Index(['well', 'rate', 'mutant'], dtype='object')


In [4]:
# for thermal 
# convert well indexes into temperatures based on map 

temp_key = pandas.read_csv( 'temperature_key.csv', index_col='Well' )
thermal['temp'] = thermal.well.str[0].map( temp_key.Celsius.to_dict() ) 

print( thermal.sample( 3 ) )

     well      rate mutant  temp
1947   C1  0.000048  T178R  44.3
928    A6  0.000112  K344A  50.0
1373   C1  0.000028  N293Q  44.3


In [5]:
# for kinetics, convert the rate into units (1/min) 
# hard-coded values:
#  0.0002 is the conversion from OD to M
#  0.25 is because we dilute the protein 4-fold upon initiation 
#  113330 is the extinction coef of BglB, used to convert mg/mL to M

kinetics[ 'kobs' ] = kinetics['rate'] * 0.0002 / ( kinetics[ 'protein_yield' ] * kinetics[ 'dilution' ] * 0.25 / 113300 )

print( kinetics.sample( 3 ) ) 

    well      rate mutant  protein_yield  dilution  note      kobs
268  E10 -0.000012  E164R           0.16       0.1   NaN -0.069113
26    C4  0.004870  N293C           1.87       0.1   NaN  2.360518
231   H5 -0.000007   S16N           0.16       0.1   NaN -0.038352


In [6]:
# also for kinetics, convert well indexes into substrate concentrations

def my_s( x ):
    my_ss = dict( zip( 'ABCDEFGH', [ .75, .1875, .046875, .01171875, .002929688, .000732422, .000183105, 0 ] ) )
    return my_ss[ x[0] ] 

kinetics[ 's' ] = kinetics.well.map( my_s ) 

In [7]:
import numpy as np 
np.mean( [ .75, .1875, .046875, .01171875, .002929688, .000732422, .000183105, 0 ] )

0.124992370625

In [8]:
# anything we need to manually drop? 

to_drop = [ 'WT', 'WT1' ] 

for df in [ kinetics, thermal ]:
    for item in to_drop:
        if item in df.index:
            df.drop( item, inplace=True ) 

In [9]:
# now export "cleaned" data 

kinetics.to_csv( '../clean/kinetics.csv' ) 
thermal.to_csv( '../clean/thermal.csv' ) 